# Custom Web Scraping and NLP Project
#### Owner: Adrianna Webb

[Github Repo](https://github.com/AdriannaWebb/web-scraping-custom)

In [7]:
# Install required packages
%pip install requests spacy spacytextblob matplotlib beautifulsoup4

# Ran 'python -m spacy download en_core_web_sm' in terminal


Note: you may need to restart the kernel to use updated packages.


In [8]:
# Import libraries
import requests
from bs4 import BeautifulSoup
from collections import Counter
import matplotlib.pyplot as plt
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import re

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Add sentiment analysis capability
nlp.add_pipe("spacytextblob")